##### **Setup**
##### Download report from BCC
1. Go to the web bank client
2. Under each currency account choose "Выписка"
3. After choosing a period -> "Отправить на почту" 


In [1]:
#imports
import pandas as pd
import re

pd.set_option('display.max_colwidth', None)

In [16]:
#parse euro_acc table
df_euro = pd.read_html('euro_acc.html')[2]
#parse tenge_acc table
df_tenge = pd.read_html('tenge_acc.html')[2]

##### Data rules:
1. All income is positive
2. All spendings are negative



##### DataFrames
1. Day-to-day transactions
2. Account transfers (incl. salary, forex, between banks)

### Preparing EURO table

In [3]:
#renaming headers
df_euro = df_euro.rename(
    columns={
        '№ п/п': 'id',
        'Дата': 'record_dt',
        'Дебет': 'sum',
        'Кредит': 'sum_temp',
        'Назначение': 'details', }
)

#changing types
df_euro['id'] = df_euro['id'].astype('int')

df_euro['sum'] = df_euro['sum'].str.replace(r'\s+', '', regex=True)
df_euro['sum'] = pd.to_numeric(df_euro['sum'])

df_euro['sum_temp'] = df_euro['sum_temp'].str.replace(r'\s+', '', regex=True)
df_euro['sum_temp'] = pd.to_numeric(df_euro['sum_temp'])

df_euro['record_dt'] = pd.to_datetime(df_euro['record_dt'], format='%d.%m.%Y')

In [4]:
#moving all sums to one column
def add_cashback(row):
    if pd.isna(row['sum']):
        row['sum'] = row['sum_temp']
    if pd.isna(row['sum_temp']):
        row['sum'] = -row['sum']
    return row

df_euro = df_euro.apply(add_cashback, axis=1)
df_euro = df_euro.drop('sum_temp', axis=1)

In [5]:
#parsing details
first_word = r'(^\w+)'
forex = r'^Покупка иностранной валюты'
atm = r'^Снятие наличных АТМ'
transfer = r'^Перевод \(списание\)'
returns = r'^Прочие зачисления на карту \(credit\)'
retail_regex = r'^(?:[^,]*,){4}([^,]*)'
retail_loc_regex = r'^[^,]*,[^,]*,\s*([^,]*,[^,]*)'
retail_datetime_regex = r'^[^,]*,\s*(\d{2}\.\d{2}\.\d{4} \d{2}:\d{2}:\d{2})'

def check_retail(row):
    if re.match(first_word, row['details']).group(0) == 'Retail':
        row['pos_loc'] = re.search(retail_loc_regex, row['details']).group(1)
        row['pos'] = re.search(retail_regex, row['details']).group(1)
        row['transaction_dt'] = re.search(retail_datetime_regex, row['details']).group(1)
        row['category_1'] = 'retail'
    elif re.match(forex, row['details']):
        row['category_1'] = 'forex'
    elif re.match(atm, row['details']):
        row['category_1'] = 'atm'
    elif re.match(transfer, row['details']):
        row['category_1'] = 'transfer'
    elif re.match(returns, row['details']):
        row['category_1'] = 'returns'
    else:
        row['category_1'] = 'unallocated'
    return row

df_euro = df_euro.apply(check_retail, axis=1)

In [6]:
# Regular expressions to extract date and retail sum directly in the loop
cashback_date_regex = r'Дата (\d{2}\.\d{2}\.\d{4} \d{2}:\d{2}:\d{2})'
retail_sum_regex = r'сумма ([\d.]+)'

# Step 2: Track indices of cashback rows that have matching retail transactions
matched_cashback_indices = []

# Step 3: Process cashback rows, find matching Retail rows, and add cashback sum
for index, row in df_euro.iterrows():
    if "Учет вознаграждений по CashBack" in row['details']:
        # Extract the cashback date and retail transaction sum directly
        cashback_date_match = re.search(cashback_date_regex, row['details'])
        retail_sum_match = re.search(retail_sum_regex, row['details'])
        
        if cashback_date_match and retail_sum_match:
            cashback_date = cashback_date_match.group(1)
            retail_amount = float(retail_sum_match.group(1))
            cashback_amount = row['sum']
            
            # Find matching Retail row by 'transaction_dt' and 'sum'
            matching_retail_index = df_euro[(df_euro['transaction_dt'] == cashback_date) & 
                                            (df_euro['sum'] == -retail_amount)].index
            
            # If match found, update cashback_sum in the Retail row and mark cashback row for deletion
            if not matching_retail_index.empty:
                df_euro.loc[matching_retail_index, 'cashback_sum'] = cashback_amount
                matched_cashback_indices.append(index)

# Step 4: Remove matched cashback rows
df_euro.drop(matched_cashback_indices, inplace=True)

# Reset index after dropping rows (optional)
df_euro.reset_index(drop=True, inplace=True)

In [7]:
# Define categories with lists of keywords for each
category_2_keywords = {
    'clothes': [
        ' COS Av.Liberdade', ' HM Grandella', ' PAYPAL *LICK.EVA', ' Hermes Lisbonne', 
        ' HAVAIANAS LISBOA', ' MASSIMO DUTTI', ' HM Colombo', 
        ' Uniqlo Europe Ltd Sucursa', ' STIVALI', ' ZARA PORTUGAL', ' INTIMISSIMI R CARMO', 
        ' SEPHORA', ' Vestiaire', 'Vinted', 'LOEWE',
    ],
    'eat_out': [
        ' CERES BOULANGERIE', ' ELE E ELA', ' COSMIKGABARITO', ' UNICO GELATO CAFFE', 
        ' BUNA SABORES', ' THE FOLKS UNIP LDA', ' BARU', ' HELLO KRISTOF', 
        ' Dallas 02 - Sao Bento', ' Janis', ' NEIGHBOURHOOD COFFEE L', 
        ' CROQUETERIA MELHORES', ' Acento coffee', ' CAFE MONKA', ' REST POMME EATERY', 
        ' BUNA', ' PIZZARIA VIAVAI', ' COMIDA INDEPENDENTE', ' Vesuviano', ' No Convento', 
        ' DEAR BREAKFAST', ' COPENHAGEN COFFEE', ' LUPITA PIZZARIA', ' CAFE LAYERS', 
        ' MARQUISE PADARIA I', ' A MANTEIGARIA', ' Fora Artisan Pastry', ' COMOBA LISBOA', 
        ' UNI', ' IMANOL PRINCIPE REAL', ' LIBERTY CAFE', ' NUMA CAFE', ' TIMEOUT', 
        ' REST TOMORROW AT 9', ' STARBUCKS DOUBLE', ' HONEST GREENS CSODRE', 
        ' REST CABO DA ROCA', ' Black Trumpet', ' Dallas 02 Sao Bento', 
        ' NEIGHBOURHOOD COFFEE', ' MARQUISE PADARIA II', ' MERCADAO', ' LANDEAU LDA', 
        ' CALMO CAFE', ' PALACIO DO GRILO LDA', ' ARCA', ' PARRA', ' CAFE SAO', 
        ' Pao do Beco', ' ACID CAFE MADRID', ' EAST CREMA COFFEE HERMOSI', 
        ' CAFETERIAS MUSEO THYSSEN', ' RESTAURANTE QUINTIN', ' BUCOLICO', ' HOT NOW', 
        ' COFI', ' STARBUCKS COFFEE ALCALA', ' Casa Neutrale', ' KIOSKO PRENSA TERESA SANC', 
        ' BOUTIQUE LINDT GOYA', ' AEROPUERTO MADRID BARAJAS', ' SQ *OSOM COFFEE', 
        ' STARBUCKS COFFEE AR', ' BAR LUCE', ' SIGNOR LIEVITO', ' Bar Transiti Imb Malpens', 
        ' Chantilly Geladaria', ' CAFE THE LAYERS', ' RESTAURANTE HOY', ' MC DONALDS OEIRAS', 
        ' YELLOW LEMON', ' DOBECO', ' BAKE BROS', ' Birkenstock Digital Gm', 
        ' ESPRESSO LAB', ' HONEST GREENS', ' PARRA WINE BISTRO', ' LA BOULANGERIE', 
        ' Isakaya by Koji', 'FABRICA', 'выдача наличных', 'OAK BERRY', 'Starbucks',
        'KIRILL IVANOV', 'RHODO BAGELS',
    ],
    'food_order': ['BOLT.EU','UBER * EATS'],
    'grocery': [
        ' FOOD MERCEARIA BIO', ' CONTINENTE BOM DIA', ' AMANHECER MINI MERC', 
        ' COMPANHIA PORTUGUEZA', ' COMP PORTUGUEZA CHA', ' LANDEAU CHIADO', 
        ' DIA FRUTA', ' MERCEARIA LUIS', ' SUPERMERCADO ESTRELA', ' ALDI PRINCIPE REAL',
        'MINIPRECO', 'Glovo', 'PINGO DOCE', 'MERCADO DE SANTOS', 'SUPER MERCADO SANTOS',
    ],
    'home': [
        ' A LINHA DA VIZINHA', ' TIGER CAMPO OURIQUE', ' SP KINFILL CAREX', 
        ' AREAS PORTUGAL SA', ' MUJI CHIADO', ' Saudade Flores', ' TIGER CHIADO', 
        ' TIGER COLOMBO', ' POPPIES DESIGN STORE', ' Aesop Cosmetics Spain', 
        ' BANEMA STUDIO LISBOA', ' ZaraHome.com', ' Flexispot GmbH',
        ' ARTURAS LUIS LDA', 'IKEA', 'FinnishDesignShopCOM', 'ZARA HOME',
    ],
    'misc': [
        ' SUMUP *SALTED BOOKS LISB', ' PCDIGA', ' EL CORTE INGLES', ' F CALOUSTE GULBENKIA', 
        ' FARMACIA CONDE BARAO', ' TICKETLINE SA', ' Farmacia Central', 
        ' CARLOS LOPES PEREIRA', ' FCALOUSTE GULBENKIAN', ' FUND CALOUSTE GULBEN', 
        ' Livraria Snob', ' FARMACIA ACOREANA', ' MAAT MUSEU ARTE', ' TERMAS DO ESTORIL', 
        ' WWW TICKETONE IT', ' CARMENCITA FILM LAB', ' CAIXAFORUM MADRID', 
        ' FUND.COLECC.THYSSEN BORNE', ' FUNDACION COLECCION THYSS', 
        ' MONDADORI BOOKSTORE GALL', ' SP ASTROPAD', ' DECATHLON LISBOA', 
        ' Liberty Books', ' LEGO CHIADO', ' FNAC LOJA DO CHIADO', 
        ' OPTICA CENTRAL CALHA', ' GAGA', ' PAPELARIA PLANETA', 'SP KEYGEM'
    ],
    'pet': [' PATINHAS MIMADAS', ' H VET SAO BENTO LDA', ' ZOOPLUS'],
    'amazon': ['Amazon', 'AMAZON', 'AMZN'],
    'subscriptions': [' BB-SAMSUNG', 'MIDJOURNEY', 'WOO', 'Netflix.com'],
    'transport': [' BIGLIETTERIA MIDATICKET', ' CLESS TICKET ATM MILANO','UBER', ],
    'travel': [
        ' EASYJET AIR K78SMJH', ' EASYJET AIR K78SGKL', ' FlyTAP', ' EASYJET AIR K7DMXW3', 
        ' AIRBNB * HMJM4T2ZX9', ' EASYJET AIR K7RM5JR', ' EASYJET 000K7RM5JR'
    ]
}



# Function to categorize based on keywords
def assign_category(row):
    for category, keywords in category_2_keywords.items():
        if any(keyword in row['details'] for keyword in keywords):
            return category
    return 'unallocated'  # Default category if no keywords match

# Apply the function to create the 'category' column
df_euro.loc[df_euro['category_1'] == 'retail', 'category_2'] = df_euro[df_euro['category_1'] == 'retail'].apply(assign_category, axis=1)


In [8]:
#check unallocated pos
df_euro[df_euro['category_2'] == 'unallocated']['pos'].unique()
#df_euro[df_euro['pos'].isna()]['details'].unique()

array([' MUSAS ARISTOCRATAS', ' BCM BRICOLAGE SA', ' ASSOCIACAO PROMOCAO',
       ' COTIDIANO COMERCIO', ' TINTURARIA SILSOL', ' MBD',
       ' Revolut**4128*', ' DICE.FM', ' BATCHI LDA', ' CASTRO - GARRETT',
       ' SUMUP *ATRAVES DO FIRMAM', ' CAMPO OURIQUE', ' IDASFEST',
       ' SISTEMA J', ' TORRE IGREJA CASTELO', ' XAFARIX', ' MEX FACTORY',
       ' RELAY VIRGIN LISBOA', ' PAUL', ' ITUGUERRA S.L.',
       ' HELADOS MAISON GLACE ESPA', ' LA ESQUINA DE RECOLETO',
       ' LA ALQUIMIA', ' GALIPPO', ' MAGPIE', " SQ *PINK'S", ' Velazquez',
       ' LEITARIA NITA', ' FNM*TRENORD TVM 2039-', ' Coop-2448 Viganell',
       ' Stazione Piccadilly SA', ' ART COMPUTER', ' IUTA BISTROT',
       ' BITRENTA SRL', ' HB SERVIZI SRL', ' FOOD TRUCK DISTRICT',
       ' BEIT EVENTS SRL', ' SUMUP *GELATERIE MILANES',
       ' T3M SNC DI ANDREA SADERI', ' GIANNASI CASSA MOBILE',
       ' SUMUP *PIAZZALE EGEO S.R', ' BLUE LION FOOD SPA',
       ' OFFICINA PROFUMO FARMACE', ' CIUMBIA SRL', ' PINACOTECA B

In [9]:
grouped_df = df_euro.groupby('category_2')['sum'].sum().reset_index()
grouped_df

#filtered_df = df_euro[df_euro['category_2'] == 'unallocated']
#grouped_df = filtered_df.groupby('pos')['sum'].sum().reset_index().sort_values(by='sum')
#grouped_df.head(30)

,category_2,sum
0,amazon,-4189.63
1,clothes,-5186.24
2,eat_out,-3751.57
3,food_order,-4501.51
4,grocery,-5920.50
5,home,-4830.10
6,misc,-5297.38
7,pet,-739.75
8,subscriptions,-498.29
9,transport,-475.20


### Preparing TENGE table

In [33]:
#renaming headers
df_tenge.columns = [' '.join(col).strip() for col in df_tenge.columns.values]

df_tenge = df_tenge.rename(
    columns={
        df_tenge.columns[0]: 'record_dt',
        df_tenge.columns[1]: 'transaction_dt',
        df_tenge.columns[2]: 'details',
        df_tenge.columns[3]: 'sum',
        df_tenge.columns[4]: 'currency',
        df_tenge.columns[5]: 'fee',
        df_tenge.columns[6]: 'total_sum',
        df_tenge.columns[7]: 'cashback',
    }
    
)
df_tenge.head(10)
#changing types
#df_euro['id'] = df_euro['id'].astype('int')

#df_euro['sum'] = df_euro['sum'].str.replace(r'\s+', '', regex=True)
#df_euro['sum'] = pd.to_numeric(df_euro['sum'])

#df_euro['sum_temp'] = df_euro['sum_temp'].str.replace(r'\s+', '', regex=True)
#df_euro['sum_temp'] = pd.to_numeric(df_euro['sum_temp'])

#df_euro['record_date'] = pd.to_datetime(df_euro['record_date'], format='%d.%m.%Y')

,record_d,transaction_dt,details,sum,currency,fee,total_sum,cashback
0,01.01.2024,01.01.2024,Перевод с карты 446375******1579 на карту 462818******4620 через систему BCC.KZ. ИИН получателя - 920404050799. Получатель - ГЕРМАН ВЛАДИМИРОВИЧ КОРЕНБЛЮМ. КНП 119 - Прочие безвозмездные переводы. Безналичный перевод. Плательщик: Самойленко Ксения Владимировна,513 400.00,KZT,0.00,513 400.00,NaN
1,01.01.2024,01.01.2024,Перевод с карты 446375******8122 на карту 462818******4620 через систему BCC.KZ. ИИН получателя - 920404050799. Получатель - ГЕРМАН ВЛАДИМИРОВИЧ КОРЕНБЛЮМ. КНП 119 - Прочие безвозмездные переводы. Безналичный перевод. Плательщик: Соколов Александр Сергеевич,308 000.00,KZT,0.00,308 000.00,NaN
2,02.01.2024,02.01.2024,Перевод с карты 446375******2183 на карту 462818******4620 через систему BCC.KZ. ИИН получателя - 920404050799. Получатель - ГЕРМАН ВЛАДИМИРОВИЧ КОРЕНБЛЮМ. КНП 119 - Прочие безвозмездные переводы. Безналичный перевод. Плательщик: Кудреватых Александр Валерьевич,225 682.00,KZT,0.00,225 682.00,NaN
3,03.01.2024,03.01.2024,"Покупка иностранной валюты клиентом розницы, Коренблюм Герман Владимирович, БИН 920404050799. Зявка №161092700. Дата вал. 03.01.2024. Покупка EUR, продажа KZT. Курс сделки 501.01. Учетный курс 502.24. Конвертация со счета KZ348562204134322019 на счет KZ938562204234322234 через систему BCC.KZ Плательщик: Коренблюм Герман Владимирович",1 062 141.20,KZT,0.00,-1 062 141.20,NaN
4,04.01.2024,04.01.2024,Перевод на счет KZ348562204134322019,67 940.00,KZT,0.00,67 940.00,NaN
5,05.01.2024,05.01.2024,"Пополнение от ТОО Яндекс.Казахстан, ИИН 170240015454, счет KZ94601A861003526971 Плательщик: ТОО Яндекс.Казахстан",1 261 538.00,KZT,0.00,1 261 538.00,NaN
6,05.01.2024,05.01.2024,"Покупка иностранной валюты клиентом розницы, Коренблюм Герман Владимирович, БИН 920404050799. Зявка №161451374. Дата вал. 05.01.2024. Покупка EUR, продажа KZT. Курс сделки 495.35. Учетный курс 500.15. Конвертация со счета KZ348562204134322019 на счет KZ938562204234322234 через систему BCC.KZ Плательщик: Коренблюм Герман Владимирович",1 312 677.50,KZT,0.00,-1 312 677.50,NaN
7,05.01.2024,05.01.2024,"BO. Годовое обслуживание счета по операциям с картой. , НДС не облагается. Плательщик: Коренблюм Герман Владимирович",10 000.00,KZT,-10 000.00,0.00,NaN
8,12.01.2024,12.01.2024,Перевод с карты 446375******5370 на счет KZ348562204134322019 через систему BCC.KZ. ИИН получателя - 920404050799. Получатель - ГЕРМАН ВЛАДИМИРОВИЧ КОРЕНБЛЮМ. КНП 119 - Прочие безвозмездные переводы. Безналичный перевод. Плательщик: Войтицкий Евгений Иванович,867 000.00,KZT,0.00,867 000.00,NaN
9,15.01.2024,12.01.2024,"Retail. Номер устройства в ПЦ, 12.01.2024 00:00:00, PRT, LISBOA, COMPANHIA PORTUGUEZA, Карта: 462818******4620 Плательщик: Коренблюм Герман Владимирович (RRN:401214454178, Auth code:531678)",42.53,USD,0.00,-18 943.29,386.60
